## CSCE 676 :: Data Mining and Analysis :: Texas A&M University :: Spring 2022


# Homework 4: The Final One!

- **100 points [7% of your final grade]**
- **Due April 28, 2022 11:59pm** 
- (*no submissions accepted after May 3, 2022 by 11:59pm*)

*Goals of this homework:* This is an **open-ended** assignment in which you will get to explore embeddings in the context of a large social media dataset. 

*Submission instructions:* You should post your notebook to canvas (look for the homework 4 assignment there). Name your submission **your-uin_hw4.ipynb**, so for example, my submission would be something like **555001234_hw4.ipynb**. Your notebook should be fully executed when you submit ... so run all the cells for us so we can see the output, then submit that. When you are done, download your notebook from colab and submit it to canvas.

*Collaboration declaration:* If you talked to anyone about this homework, please be sure to mention that. Remember to include citations to any sources you use in the homework.

*Write your collaboration/references here*

## Instructions: 

For this homework, you will conduct an independent data mining analysis using embeddings of the congressional Twitter dataset we used in Homework 2. The requirements are threefold:

*   You must use some (or all) of the US Congress Twitter data we provided in Homework 2. You may augment the data if you like, but that is not required. Also feel free to sample just a portion of the data (you do not have to use all of it).
*   You must learn embeddings over the dataset. You may use word2vec to find word embeddings, node2vec to find node embeddings, or some other dense representation. It is up to you, but you must learn some kind of embedding. You may use an existing package to learn these embeddings. Feel free to google around to find the right resource. Some examples include: 

  * word2vec (python, in Gensim) https://radimrehurek.com/gensim/models/word2vec.html
  * doc2vec (python, in Gensim) https://radimrehurek.com/gensim/models/doc2vec.html
  * Node2Vec (python) https://github.com/aditya-grover/node2vec
  * LINE (C++) https://github.com/tangjianpku/LINE
  * DeepWalk (python) https://github.com/phanein/deepwalk, https://pypi.org/project/deepwalk/

* Using these embeddings, you must conduct an interesting analysis of the dataset. For example, are there communities of users you can discover in the node embedding space? Are there outliers in the word embedding space? This is the most interesting part, and you should do your best to make this compelling.


### In the following, you should answer the questions we pose. Then below, you should include your code, including helpful comments and discussion so we can follow your logic.




##  Question 1

What is your compelling data mining application? That is, what is it you aim to discover?

My aim is use most frequent hashtags to find similar trending hashtags, meaningful words and congressman and for twitter users.




##  Question 2

What data did you use? Describe briefly what aspects of the US Congressional data you used.

I have used US congress 400k tweets dataset.
1. user tweets : text
2. user mentions : entities.user_mentions
3. user's names : user.screen_names
4. hashtags : user.hashtags
 



##  Question 3

What embedding method did you use? And why?

I have used word2vec because it helps to map relations between popular hastags, tweet words and user mentions.
I have kept '@' before user_mentions and kept user's names between curly braces to identilfy them when finding similar items.




##  Question 4

How did you apply your embeddings to tackle the problem you posed?

First i have only kept meaningful words from tweets by removing stop-words, punctuations etc and then added user_mentions int the form of "@user" and then added users who tweeted the tweet in the form of "{user_name}" and later tokenized them.



##  Question 5

What did you discover? Provide your results and analysis here.


While finding similar items with hashtags, model showed relevant trending hastags, users and important key words. 

For example for **'#Venezuela'** the similar words shown include **#caracas**, **@freddysuperlano** who is venezulan politician etc.

**Other important observations**
Words that are not in vocabulary of model cannot be used to find similarity.
It relies on similar language words


# Your Analysis

In [1]:
!pip install pyspark
!pip install graphframes
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!curl -L -o "/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.1-spark3.0-s_2.12.jar" https://repos.spark-packages.org/graphframes/graphframes/0.8.1-spark3.0-s_2.12/graphframes-0.8.1-spark3.0-s_2.12.jar
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  242k  100  242k    0     0  1908k      0 --:--:-- --:--:-- --:--:-- 1908k


In [2]:
import pandas as pd
import numpy as np
!mkdir us-congress-tweets
!wget https://us-congress.s3.amazonaws.com/congress_members.csv -O us-congress-tweets/congress_members.csv
!wget https://us-congress.s3.amazonaws.com/tweets-sample-400k.json -O us-congress-tweets/tweets-sample-400k.json
!wget https://us-congress.s3.amazonaws.com/user_hashtags-1m.csv -O us-congress-tweets/user_hashtags-1m.csv
!wget https://us-congress.s3.amazonaws.com/user_mentions.csv -O us-congress-tweets/user_mentions.csv

mkdir: cannot create directory ‘us-congress-tweets’: File exists
--2022-05-01 23:50:05--  https://us-congress.s3.amazonaws.com/congress_members.csv
Resolving us-congress.s3.amazonaws.com (us-congress.s3.amazonaws.com)... 52.216.110.187
Connecting to us-congress.s3.amazonaws.com (us-congress.s3.amazonaws.com)|52.216.110.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13835 (14K) [text/csv]
Saving to: ‘us-congress-tweets/congress_members.csv’

us-congress-tweets/ 100%[===================>]  13.51K  --.-KB/s    in 0s      

2022-05-01 23:50:05 (112 MB/s) - ‘us-congress-tweets/congress_members.csv’ saved [13835/13835]

--2022-05-01 23:50:06--  https://us-congress.s3.amazonaws.com/tweets-sample-400k.json
Resolving us-congress.s3.amazonaws.com (us-congress.s3.amazonaws.com)... 52.216.110.187
Connecting to us-congress.s3.amazonaws.com (us-congress.s3.amazonaws.com)|52.216.110.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 202495914 

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark import SparkContext, SparkConf


In [4]:
# create a Spark session
conf = SparkConf().set("spark.ui.port", "4050")

# create a Spark context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [18]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
twitter_date_format="EEE MMM dd HH:mm:ss ZZZZZ yyyy"

user_schema = StructType([
    StructField('created_at', TimestampType(), True),
    StructField('followers_count', LongType(), True),
    StructField('id', LongType(), True),
    StructField('name', StringType(), True),
    StructField('screen_name', StringType(), True)
])

hashtag_schema = ArrayType(StructType([StructField('text', StringType(), True)]))
user_mentions_schema = ArrayType(StructType([StructField('id', LongType(), True),
                                             StructField('screen_name', StringType(), True)]))
entities_schema = StructType([
    StructField('hashtags', hashtag_schema, True),
    StructField('user_mentions', user_mentions_schema, True)
    ])

retweeted_status_schema =StructType([        
        StructField("id", LongType(), True),
        StructField("in_reply_to_user_id", LongType(), True),
        StructField("in_reply_to_status_id", LongType(), True),
        StructField("created_at", TimestampType(), True),
        StructField("user", user_schema)
    ])

tweet_schema =StructType([
        StructField("text", StringType(), True),
        StructField("id", LongType(), True),
        StructField("in_reply_to_user_id", LongType(), True),
        StructField("in_reply_to_status_id", LongType(), True),
        StructField("created_at", TimestampType(), True),
        StructField("user", user_schema),
        StructField("entities", entities_schema),
        StructField("retweeted_status", retweeted_status_schema)
    ])

tweets = spark.read.option("timestampFormat", twitter_date_format)\
                   .json('us-congress-tweets/tweets-sample-400k.json', tweet_schema)\
                   .withColumn('user_id',F.col('user.id'))
tweets.printSchema()

root
 |-- text: string (nullable = true)
 |-- id: long (nullable = true)
 |-- in_reply_to_user_id: long (nullable = true)
 |-- in_reply_to_status_id: long (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- created_at: timestamp (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- screen_name: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- user_mentions: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- screen_name: string (nullable = true)
 |-- retweeted_status: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- in_reply_to_user_id: long (nul

In [33]:
#tweets_df = tweets.toPandas()
from pyspark.sql.functions import col


tweets_df = tweets.select(col("text"), col("user.screen_name"),col("entities.user_mentions.screen_name").alias("user_mentions"), col("entities.hashtags.text").alias("hashtags"))
tweets_df = tweets_df.toPandas()

In [34]:
tweets_df


,text,screen_name,user_mentions,hashtags
0,"RT @maddow: ""I hear a lot about about lack of ...",quazy101,[maddow],[]
1,@ChuckGrassley @SenateMajLdr It's time for you...,sparadiso11,"[ChuckGrassley, senatemajldr]",[]
2,RT @cm_merlin: @funder @JeffFlake @lisamurkows...,disqualify_gop,"[cm_merlin, funder, JeffFlake, lisamurkowski, ...",[]
3,RT @SenJeffMerkley: I read the FBI report. Th...,c_millerhorton,[SenJeffMerkley],[]
4,RT @SenJeffMerkley: I read the FBI report. Th...,philipsgma,[SenJeffMerkley],[]
...,...,...,...,...
399995,@tribelaw @RepRaskin @RepAdamSchiff @RepJerryN...,2know,"[tribelaw, RepRaskin, RepAdamSchiff, RepJerryN...",[]
399996,RT @RepMarkMeadows: A beyond disgraceful state...,pigletjay,[RepMarkMeadows],[]
399997,@Bilbre1 @SenBlumenthal @IvankaTrump Always wi...,AreYouIn2,"[Bilbre1, SenBlumenthal, IvankaTrump]",[]
399998,RT @GiffordsCourage: Summer weekends should be...,pkcampsen,[GiffordsCourage],[]


In [35]:
import re 

tweets_df['c_text'] = tweets_df['text'].str.replace(r'@[A-Za-z0-9]+', '',regex=True) #remove handles
tweets_df['c_text'] = tweets_df['c_text'].replace('RT', '', regex=True) #remove word before colon
tweets_df['c_text'] = tweets_df['c_text'].str.replace("[^a-zA-Z#]", " ") #remove punctuations

tweets_df['c_text'] = tweets_df['c_text'].str.replace("RT", "")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [36]:
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
tweets_df = tweets_df.dropna(axis=0, subset=['c_text'])
tweets_df['c_text'] = tweets_df['c_text'].apply(lambda x: [item for item in x.split() if item not in stop])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [37]:

tweets_df['user_mentions'] = tweets_df['user_mentions'].apply(lambda x:['@' + item for item in x]) # @ represents mentions
tweets_df['hashtags'] = tweets_df['hashtags'].apply(lambda x:['#' + item for item in x])
tweets_df['screen_name'] = '{' +  tweets_df['screen_name'] + '}' #bracket shows the name of the user

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [40]:
tweets_df['c_text'] = tweets_df['c_text'] + tweets_df['user_mentions'] + tweets_df['hashtags']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [41]:
for i in tweets_df.index:
  tweets_df['c_text'][i].append(tweets_df['screen_name'][i])


In [42]:

tokenize = tweets_df['c_text']

In [43]:
import gensim
from sklearn.model_selection import train_test_split
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
w2vec_model= Word2Vec(tokenize,min_count = 1, size = 100, window = 5, sg = 1)
w2vec_model.train(tokenize,total_examples= len(tweets_df['text']),epochs=20)


(75585122, 79906580)

In [50]:

frequent_hashtags = tweets_df['hashtags'].value_counts()[:6].index.tolist()
# user_mentions = ['@NancyPelosi','@LindseyGrahamSc','@SenSchumer','@RepAdamSchiff']
# # for user in user_mentions:
# #    print("Top 10 similar items to " + user + " : " + str(w2vec_model.most_similar(user, topn=10))[1:-1])
# print(w2vec_model.most_similar('@NancyPelosi', topn=10))
# print(w2vec_model.most_similar('@SenSchumer', topn=10))
# print(w2vec_model.most_similar('@RepAdamSchiff', topn=10))

In [53]:

frequent_hashtags = [ele for ele in frequent_hashtags if ele != []]
frequent_hashtags

[['#Venezuela'],
 ['#MoscowMitch'],
 ['#TrumpShutdown'],
 ['#MuellerReport'],
 ['#GreenNewDeal']]

In [57]:
print(w2vec_model.wv.most_similar('#Venezuela', topn=5))
print(w2vec_model.wv.most_similar('#MoscowMitch', topn=5))
print(w2vec_model.wv.most_similar('#TrumpShutdown', topn=5))
print(w2vec_model.wv.most_similar('#MuellerReport', topn=5))
print(w2vec_model.wv.most_similar('#GreenNewDeal', topn=5))


[('#Caracas', 0.822726309299469), ('#Maduro', 0.8165556192398071), ('#MaduroRegime', 0.8089548349380493), ('#Colombia', 0.7877011299133301), ('@freddysuperlano', 0.7836524844169617)]
[('#MoscowMitchTraitor', 0.7722510099411011), ('#MoscowMitchMcTraitor', 0.7438052892684937), ('#MassacreMitch', 0.7416695356369019), ('#MoscowMitchIsCorrupt', 0.7376363277435303), ('#MoscowMitchMcTreason', 0.7309906482696533)]
[('#MitchMcConnellShutdown', 0.7082730531692505), ('#FakeCrisis', 0.6943866014480591), ('#TrumpFactCheck', 0.6915230751037598), ('#ChronicLiar', 0.6905205249786377), ('#DistractorInChief', 0.6876767873764038)]
[('#TweetTheMuellerReport', 0.7014746069908142), ('LlwtCb', 0.6853803396224976), ('#ColusionDelusion', 0.683164119720459), ('pylV', 0.6779931783676147), ('#JustImpeach', 0.6736583709716797)]
[('@sunrisemvmt', 0.677780270576477), ('disres', 0.659604549407959), ('@DSA_Enviro', 0.6444772481918335), ('#AOCCult', 0.6392322778701782), ('pUKZOHGIst', 0.6391947865486145)]
